In [ ]:
!pip install gradio openai vllm bitsandbytes




Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:

!vllm serve thuanan/Llama-3.2-1B-RLHF-2k-vi-alpaca \
 --api-key "vodaotuandat" \
 --compilation-config '{"cache_dir": "../cache"}' \
 --port 8000 \
 --quantization bitsandbytes \
 --enable-prefix-caching \
 --swap-space 8 \
 --gpu-memory-utilization 0.9 \
 --disable-log-requests \
 --max-model-len 2048

In [ ]:
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8000/v1",   # ĐẶT /v1 ở cuối
    api_key="vodaotuandat"
)

resp = client.chat.completions.create(
    model="thuanan/Llama-3.2-1B-RLHF-2k-vi-alpaca",
    messages=[{"role":"user","content":"Sự nghiệp của Cristiano RonalDo"}],
)
print(resp.choices[0].message.content)

Cristiano Ronaldo, người chơi bóng nổi tiếng từ Portugal, có sự nghiệp đầy thăng trầm và thành công. Dưới đây là một số điểm chính của sự nghiệp của anh ấy:

- Đầu tiên, Ronaldo là một cầu thủ bóng đá chuyên nghiệp, bắt đầu sự nghiệp của mình tại câu lạc bộ bóng đá Juventus (nay là Juventus) ở Italy vào năm 2002. Anh nhanh chóng trở thành một trong những cầu thủ trẻ nhất của câu lạc bộ này.

- Vào năm 2003, Ronaldo chuyển sang Manchester United, nơi anh chơi 5 năm trước khi chuyển sang Sporting CP (nay là Sporting CP) ở Portugal vào năm 2005. Anh trở thành một trong những cầu thủ hàng đầu của câu lạc bộ này.

- Năm 2009, Ronaldo chuyển sang Real Madrid, nơi anh chơi 7 năm trước khi chuyển sang Juventus vào năm 2018. Câu lạc bộ này đã đưa anh đến thành công, giành 5 giải Champions League, 4 giải UEFA Cup, 1 giải Copa America và 1 giải Copa Libertadores.

- Vào năm 2015, Ronaldo chuyển sang Manchester United lại, nơi anh chơi 1 năm trước khi chuyển sang Juventus vào năm 2018. Juventus đã

In [4]:



# ──────────────────────────────────────────────
# 1) KHAI BÁO HẰNG SỐ – sửa tại đây
OPENAI_API_KEY   = "vodaotuandat"                           # hoặc chuỗi tuỳ ý
OPENAI_API_BASE  = "http://127.0.0.1:8000/v1"               # vLLM endpoint nội bộ
CHAT_MODEL       = "thuanan/Llama-3.2-1B-RLHF-2k-vi-alpaca" # tên model
# ──────────────────────────────────────────────

import gradio as gr, base64, os, io
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY, base_url=OPENAI_API_BASE)


# ──────────── TIỆN ÍCH GIAO DIỆN ────────────
def image_to_base64(path: str) -> str:
    try:
        with open(path, "rb") as f:
            return base64.b64encode(f.read()).decode()
    except FileNotFoundError:
        return ""

def get_header_html() -> str:
    logo_html = "<h2>ChatBot VIETNAM</h2>"
    if os.path.exists("static/logo.png"):
        b64 = image_to_base64("static/logo.png")
        logo_html = f'<img src="data:image/png;base64,{b64}" style="height:120px;">'

    return f"""
    <div style="display:flex;align-items:center;padding:20px 0;">
      {logo_html}
      <div style="margin-left:20px">
        <h1>🧠 LLM Chatbot</h1>
        <p>Conversational Assistant • Vietnamese Chatbot</p>
      </div>
    </div>"""

def get_footer_html() -> str:
    return """
    <div style="
        position:fixed;bottom:0;left:0;width:100%;
        background:#fff;padding:8px;text-align:center;
        box-shadow:0 -2px 8px rgba(0,0,0,.1);font-size:14px">
    </div>"""


# ──────────── HÀM CHAT ────────────
def respond(message, history, system_msg, max_tokens, temperature, top_p):
    # history = list[dict] theo format gradio (“messages”)
    msgs = [{"role":"system","content":system_msg}] + history + \
           [{"role":"user","content":message}]

    stream = client.chat.completions.create(
        model       = CHAT_MODEL,
        messages    = msgs,
        max_tokens  = max_tokens,
        temperature = temperature,
        top_p       = top_p,
        stream      = True,
    )

    answer = ""
    for chunk in stream:
        delta = chunk.choices[0].delta
        if delta.content:
            answer += delta.content
            yield answer          # chỉ chuỗi là đủ


# ──────────── GRADIO UI ────────────
chat = gr.ChatInterface(
    fn   = respond,
    type = "messages",
    additional_inputs = [
        gr.Textbox(value="Bạn là một Chatbot Tiếng Việt.", label="System message"),
        gr.Slider(1, 2048, value=512, step=1, label="Max new tokens"),
        gr.Slider(0.1, 4.0, value=0.7, step=0.1, label="Temperature"),
        gr.Slider(0.1, 1.0, value=0.95, step=0.05, label="Top-p (nucleus)")
    ],
)

with gr.Blocks(css=".gradio-container {min-height:100vh;}") as demo:
    gr.HTML(get_header_html())
    chat.render()
    gr.HTML(get_footer_html())

# ──────────── CHẠY ────────────
os.makedirs("static", exist_ok=True)  # nếu cần chỗ đặt logo
demo.launch(server_name="0.0.0.0", share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://43098c1fc802ade8a5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')